In [1]:
!pip install flask pyngrok diffusers transformers accelerate safetensors

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 63.9 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.9.41
    Uninstalling nvidia-nvjitlink-cu12-12.9.41:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.9.41
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.10.19
    Uninstalling nvidia-curand-cu12-

In [2]:
!ngrok authtoken '2j9anopFIcJFDQnikEfB9tlTU3z_3UGjUMEjP1cadx8fBi3Mr'

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml                                


In [3]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
from diffusers import StableDiffusionPipeline
import torch
import os
from PIL import Image
import uuid

app = Flask(__name__)

# Ensure the 'static' directory exists
if not os.path.exists('static'):
    os.makedirs('static')

# Load the Stable Diffusion model
print("Loading model...")
pipe = StableDiffusionPipeline.from_pretrained(
    "CompVis/stable-diffusion-v1-4",
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
)
pipe = pipe.to("cuda" if torch.cuda.is_available() else "cpu")
pipe.safety_checker = lambda images, **kwargs: (images, [False] * len(images))
print("Model loaded successfully!")

@app.route("/api/generate", methods=["POST"])
def generate():
    data = request.get_json()
    prompt = data.get("prompt")
    if not prompt:
        return jsonify({"error": "No prompt provided"}), 400

    with torch.autocast("cuda" if torch.cuda.is_available() else "cpu"):
        image = pipe(prompt).images[0]

    # Save the image with a unique filename
    filename = f"{uuid.uuid4().hex}.png"
    image_path = os.path.join("static", filename)
    image.save(image_path)

    # Construct the public URL
    public_url = ngrok.connect(5000)
    image_url = f"{public_url}/static/{filename}"

    return jsonify({
        "prompt": prompt,
        "image_url": image_url
    })


2025-06-02 16:51:46.726346: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748883106.916628      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748883106.975445      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Loading model...


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

preprocessor_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

vae/diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

scheduler_config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Model loaded successfully!


In [4]:
# 1. Install diffusers and transformers if not already installed
!pip install -q diffusers transformers accelerate

# 2. Import libraries
from diffusers import StableDiffusionPipeline
import torch
import os

# 3. Load Stable Diffusion pipeline (fp16 for speed if GPU is available)
model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32)
pipe = pipe.to("cuda" if torch.cuda.is_available() else "cpu")

# 4. Generate image
prompt = "a futuristic cityscape at sunset, ultra-detailed, concept art"
image = pipe(prompt).images[0]

# 5. Save generated image
output_dir = "stable_diffusion_output"
os.makedirs(output_dir, exist_ok=True)
#image_path = os.path.join(output_dir, "generated_image.png")
#image.save(image_path)

# 6. Save the entire model
pipe.save_pretrained(os.path.join(output_dir, "saved_model"))

print(f"Image and model saved in: {output_dir}")


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

scheduler_config.json:   0%|          | 0.00/308 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.72k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

...

diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/50 [00:00<?, ?it/s]

Image and model saved in: stable_diffusion_output


In [5]:
!pip install flask-tunnel


  Preparing metadata (setup.py) ... done
  Created wheel for flask-tunnel: filename=flask_tunnel-0.1.5-py3-none-any.whl size=2952 sha256=77b134f9bc3251be8793a1aed5a86a3cc053e409aea3dc532ffb029382ee0d91
  Stored in directory: /root/.cache/pip/wheels/d5/c0/3a/3b7323db27da73268b14b67c3ea65233e8ac4d9827226885bd
Successfully built flask-tunnel


In [6]:
!ngrok config add-authtoken ts_2xodFPnCy966BohIW6VL1Jn1M7x

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [7]:
from pyngrok import conf

# Set ngrok authtoken (replace with your actual token)
conf.get_default().auth_token = "2xxDqFPRGhFd2IvDOFRKHtgvORL_41Xu8WPj2g5m44F4KpCei"


In [ ]:
# If not already installed, uncomment below to install required packages:
!pip install flask flask_cors pyngrok diffusers torch pillow --quiet

from flask import Flask, request, jsonify
from flask_cors import CORS
from PIL import Image
import torch
from diffusers import StableDiffusionPipeline
import base64
from io import BytesIO
from pyngrok import ngrok
import time

# Initialize Flask app
app = Flask(__name__)
CORS(app)  # Enable CORS for all routes (for development use)

# Load Stable Diffusion model
model_path = "/kaggle/working/stable_diffusion_output/saved_model"
pipe = StableDiffusionPipeline.from_pretrained(model_path, torch_dtype=torch.float32)
pipe = pipe.to("cuda" if torch.cuda.is_available() else "cpu")

@app.route("/")
def home():
    return "✅ Flask backend is up and running!"

@app.route("/api/generate", methods=["POST"])
def generate_image():
    print("Request headers:", request.headers)
    print("Raw request data:", request.data)

    try:
        data = request.get_json(force=True)
        print("Parsed JSON data:", data)
    except Exception as e:
        print("Error parsing JSON:", e)
        return jsonify({"error": "Invalid JSON format."}), 400

    prompt = data.get("prompt", "A beautiful landscape")

    try:
        with torch.no_grad():
            image = pipe(prompt).images[0]

        # Convert image to base64
        buffered = BytesIO()
        image.save(buffered, format="PNG")
        img_b64 = base64.b64encode(buffered.getvalue()).decode("utf-8")

        return jsonify({
            "prompt": prompt,
            "image_b64": img_b64  # Frontend will render this using <img src="data:image/png;base64,..." />
        })

    except Exception as e:
        print(f"Error generating image: {e}")
        return jsonify({"error": "Image generation failed."}), 500

if __name__ == "__main__":
    port = 5001

    # Kill any existing tunnels
    ngrok.kill()

    # Start new ngrok tunnel
    public_url = ngrok.connect(port).public_url
    print(f"✅ Public URL: {public_url}")

    # Start Flask app
    app.run(port=port)


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

✅ Public URL: https://4f71-34-66-249-221.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off
Request headers: Host: 4f71-34-66-249-221.ngrok-free.app
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/137.0.0.0 Safari/537.36
Content-Length: 76
Accept: */*
Accept-Encoding: gzip, deflate, br, zstd
Accept-Language: en-US,en;q=0.8
Content-Type: application/json
Origin: http://localhost:3000
Priority: u=1, i
Referer: http://localhost:3000/
Sec-Ch-Ua: "Brave";v="137", "Chromium";v="137", "Not/A)Brand";v="24"
Sec-Ch-Ua-Mobile: ?0
Sec-Ch-Ua-Platform: "Windows"
Sec-Fetch-Dest: empty
Sec-Fetch-Mode: cors
Sec-Fetch-Site: cross-site
Sec-Gpc: 1
X-Forwarded-For: 2409:40f2:2f:b0fc:acbb:f183:41aa:45ae
X-Forwarded-Host: 4f71-34-66-249-221.ngrok-free.app
X-Forwarded-Proto: https


Raw request data: b'{"prompt":"A photo of a forest trail with sunlight filtering through trees"}'
Parsed JSON data: {'prompt': 'A photo of a forest trail with sunl

  0%|          | 0/50 [00:00<?, ?it/s]

Request headers: Host: 4f71-34-66-249-221.ngrok-free.app
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/137.0.0.0 Safari/537.36
Content-Length: 68
Accept: */*
Accept-Encoding: gzip, deflate, br, zstd
Accept-Language: en-US,en;q=0.8
Content-Type: application/json
Origin: http://localhost:3000
Priority: u=1, i
Referer: http://localhost:3000/
Sec-Ch-Ua: "Brave";v="137", "Chromium";v="137", "Not/A)Brand";v="24"
Sec-Ch-Ua-Mobile: ?0
Sec-Ch-Ua-Platform: "Windows"
Sec-Fetch-Dest: empty
Sec-Fetch-Mode: cors
Sec-Fetch-Site: cross-site
Sec-Gpc: 1
X-Forwarded-For: 2409:40f2:2f:b0fc:acbb:f183:41aa:45ae
X-Forwarded-Host: 4f71-34-66-249-221.ngrok-free.app
X-Forwarded-Proto: https


Raw request data: b'{"prompt":"A clear photo of a snowy mountain peak under blue skies"}'
Parsed JSON data: {'prompt': 'A clear photo of a snowy mountain peak under blue skies'}


  0%|          | 0/50 [00:00<?, ?it/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

Request headers: Host: 4f71-34-66-249-221.ngrok-free.app
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/137.0.0.0 Safari/537.36
Content-Length: 68
Accept: */*
Accept-Encoding: gzip, deflate, br, zstd
Accept-Language: en-US,en;q=0.8
Content-Type: application/json
Origin: http://localhost:3000
Priority: u=1, i
Referer: http://localhost:3000/
Sec-Ch-Ua: "Brave";v="137", "Chromium";v="137", "Not/A)Brand";v="24"
Sec-Ch-Ua-Mobile: ?0
Sec-Ch-Ua-Platform: "Windows"
Sec-Fetch-Dest: empty
Sec-Fetch-Mode: cors
Sec-Fetch-Site: cross-site
Sec-Gpc: 1
X-Forwarded-For: 2409:40f2:2f:b0fc:acbb:f183:41aa:45ae
X-Forwarded-Host: 4f71-34-66-249-221.ngrok-free.app
X-Forwarded-Proto: https


Raw request data: b'{"prompt":"A clear photo of a snowy mountain peak under blue skies"}'
Parsed JSON data: {'prompt': 'A clear photo of a snowy mountain peak under blue skies'}


  0%|          | 0/50 [00:00<?, ?it/s]

Request headers: Host: 4f71-34-66-249-221.ngrok-free.app
User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/137.0.0.0 Safari/537.36
Content-Length: 80
Accept: */*
Accept-Encoding: gzip, deflate, br, zstd
Accept-Language: en-US,en;q=0.8
Content-Type: application/json
Origin: http://localhost:3000
Priority: u=1, i
Referer: http://localhost:3000/
Sec-Ch-Ua: "Brave";v="137", "Chromium";v="137", "Not/A)Brand";v="24"
Sec-Ch-Ua-Mobile: ?0
Sec-Ch-Ua-Platform: "Windows"
Sec-Fetch-Dest: empty
Sec-Fetch-Mode: cors
Sec-Fetch-Site: cross-site
Sec-Gpc: 1
X-Forwarded-For: 2409:40f2:2f:b0fc:acbb:f183:41aa:45ae
X-Forwarded-Host: 4f71-34-66-249-221.ngrok-free.app
X-Forwarded-Proto: https


Raw request data: b'{"prompt":"\\"A realistic photo of a sunset over the ocean with seagulls flying"}'
Parsed JSON data: {'prompt': '"A realistic photo of a sunset over the ocean with seagulls flying'}


  0%|          | 0/50 [00:00<?, ?it/s]